In [44]:
!pip install requests-html

In [45]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
import json
import csv
import datetime

In [46]:
def GetCategoriesIDList():
  categories_link = ['https://tiki.vn/dien-tu-dien-lanh/c4221']
  category_id_list = []
  category_url_key = []
  sess = HTMLSession()
  for link in categories_link:
    category_html = sess.get(link)
    category_soup = BeautifulSoup(category_html.content)
    sub_categories_html = category_soup.findAll('a',class_='item item--category ')
    for sub_category_link in sub_categories_html:
      item = sub_category_link.attrs["href"]
      categories_link.append(item)
  for i in range(0,len(categories_link)):
    category_id = categories_link[i].split('/')[4].split('c')[1]
    category_id_list.append(category_id)
    url_key = categories_link[i].split('/')[3]
    category_url_key.append(url_key)
  return category_id_list,category_url_key

In [47]:
id_list,url_key = GetCategoriesIDList()

In [48]:
i = 1
k=0
product_id_list = []
sess = HTMLSession()
category_id_list,category_url_key= GetCategoriesIDList()
page_api = 'https://tiki.vn/api/v2/products?category={}&page={}&urlKey={}'
while (k < len(category_id_list)):
  page_json = sess.get(page_api.format(category_id_list[k],str(i),category_url_key[k]))
  noidung = json.loads(page_json.text)
  is_done = len(noidung['data'])
  if (is_done == 0): 
      k+=1 
      i=1
  for j in range(0,len(noidung['data'])):
    id = noidung['data'][j]['id']
    if not id in product_id_list:
      product_id_list.append(id)
  page_json.close()
  noidung.clear()
  i+=1

In [49]:
product_id_list

[50702921,
 50421559,
 50702927,
 11104563,
 17465711,
 35933885,
 68832412,
 548549,
 10003165,
 562471,
 12743825,
 16033028,
 7749111,
 50590611,
 47215212,
 29245829,
 50702919,
 4287801,
 33740550,
 50702917,
 14934073,
 15239910,
 17493952,
 4374799,
 845378,
 14934071,
 17493954,
 556302,
 29240786,
 3665301,
 11656299,
 19552942,
 36457818,
 44183140,
 1733725,
 4301925,
 53646228,
 49089541,
 29240829,
 44065922,
 50421565,
 23556604,
 15239985,
 58292845,
 17487126,
 65439712,
 50425707,
 68832410,
 29247864,
 65453428,
 15647659,
 41098424,
 12724684,
 68832414,
 50560441,
 486701,
 1353567,
 59249458,
 3818757,
 4191389,
 35580965,
 15647648,
 548535,
 50692299,
 11610594,
 52696677,
 50421561,
 50421562,
 543062,
 54181975,
 1676159,
 20200144,
 54181973,
 1311789,
 11104347,
 52702602,
 58629879,
 25933434,
 50189662,
 50702925,
 732475,
 756761,
 50692303,
 11714660,
 51235932,
 50692301,
 50590609,
 5983519,
 11593167,
 536240,
 548537,
 11616101,
 504683,
 3819821,
 89

In [50]:
len(product_id_list)

7330

In [51]:
def FindInfor(sp,name):
  for i in range(0,len(sp["specifications"][0]["attributes"])):
    if (sp["specifications"][0]["attributes"][i]["name"] == name):
      return i
  return '####'

In [52]:
def GetRow(id):
  sess = HTMLSession()
  product_api = 'https://tiki.vn/api/v2/products/{}'
  product_json = sess.get(product_api.format(id))
  sp = json.loads(product_json.content)
  description = BeautifulSoup(sp["description"])
  root = ' ####' 
  brand_root = '#### '
  brand = '####'
  root_index = FindInfor(sp,"Xuất xứ")
  brand_root_index = FindInfor(sp,"Xuất xứ thương hiệu")
  if (root_index != '####'): 
    root = sp["specifications"][0]["attributes"][root_index]["value"]
  if (brand_root_index != '####'): 
    brand_root = sp["specifications"][0]["attributes"][brand_root_index]["value"]
  if  "brand" in sp:
    brand = sp["brand"]["name"]
  row = [sp["id"], sp["name"], 
              brand, root, 
              brand_root, sp["sku"],
              sp["list_price"],sp["discount_rate"],sp["price"],
              description.text, sp["productset_group_name"]]
  return row


In [53]:
row = GetRow(24787086)
print(row)

[24787086, 'Smart Tivi LG Full HD 43 inch 43LM5700PTC', 'LG', 'Indonesia', 'Hàn Quốc', '6146318198535', 9900000, 34, 6579000, 'Loại Tivi:\tSmart Tivi, 43 inch \nĐộ phân giải: Full HD\nTần số quét: 50Hz\nHệ điều hành: WebOS 4.5\nỨng dụng có sẵn: Youtube, Netflix, Trình duyệt web, LG Content Store\nỨng dụng có thể tải thêm : FPT Play, ClipTV, Fim+, MyTV, Nhaccuatui, HTV, Spotify, Zing TV...\nCông nghệ hình ảnh: Chip xử lý thông minh Quad Core Processor, HDR Effect, Active HDR, Nâng cấp độ phân giải (Resolution Upscaler)Thiết kế đơn giản, tinh tế\nSmart Tivi LG Full HD 43 inch 43LM5700PTC\xa0có thiết kế viền mỏng và phong cách, tạo nên sự hài hòa với nội thất của bạn để tạo ra một trải nghiệm xem tốt hơn.\nTivi LG 43 inch\xa0phù hợp với các không gian vừa và nhỏ như phòng khách, phòng ngủ,\n\nĐộ phân giải\xa0Full HD\nTivi LG\xa0này trang bị độ phân giải Full HD sắc nét gấp 2 lần tivi HD cùng với công nghệ Resolution Upscaler giúp nâng cấp những hình ảnh có độ phân giải thấp lên gần bằng c

In [54]:
def SaveProductToFile(id_list):
  p= open("product.csv",encoding="utf8",mode="w",)
  csv_writer = csv.writer(p)
  csv_writer.writerow(['MaSP','TenSP','ThuongHieu','XuatXu','XuatXuThuongHieu','SKU','GiaGoc','PhanTramKM','GiaKM','MoTa','PhanLoai'])
  size = len(id_list)
  for i in range(0,size):
      try: 
        row = GetRow(id_list[i])
      except:
        print("Lỗi sản phẩm: ", id_list[i])
        print("Retrying....")
        row = GetRow(id_list[i])
        print(row)
      csv_writer.writerow(row)
  return size

In [55]:
def SaveCommentToFile(id_list):
  numCmt = 0
  sess = HTMLSession()
  customer_list = []
  customer_id_list = []
  comment_api = 'https://tiki.vn/api/v2/reviews?product_id={}'
  p = open("comment.csv",encoding="utf8",mode="w",)
  csv_writer = csv.writer(p)
  csv_writer.writerow(['MaBinhLuan','MaKhachHang','MaSP','NoiDung','Rating','ThoiGian'])
  for id_product in id_list:
        comment_json = sess.get(comment_api.format(id_product))
        cmt = json.loads(comment_json.content)
        if (len(cmt) > 0 ):
            for i in range(0,len(cmt['data'])):
              thoigian = datetime.datetime.fromtimestamp(int(cmt['data'][i]['created_at'])).strftime('%Y-%m-%d %H:%M:%S')
              row = [cmt['data'][i]['id'], cmt['data'][i]['customer_id'],id_product, cmt['data'][i]['content'], cmt['data'][i]['rating'], thoigian]
              csv_writer.writerow(row)
              if not cmt['data'][i]['customer_id'] in customer_id_list:
               customer_id_list.append(cmt['data'][i]['customer_id'])
               customer = [cmt['data'][i]['customer_id'],cmt['data'][i]['created_by']['name']]
               customer_list.append(customer)
              numCmt+=1
  return numCmt,customer_list

In [56]:
def SaveCustomerToFile(id_list):
  p = open("customer.csv",encoding="utf8",mode = "w")
  csv_writer = csv.writer(p)
  csv_writer.writerow(['MaKhachHang','TenKH'])
  for cus in id_list:
    csv_writer.writerow(cus)
  return len(id_list)


In [ ]:
record = SaveProductToFile(product_id_list)

In [58]:
print(record)

7330


In [59]:
line,customer_list = SaveCommentToFile(product_id_list)

In [60]:
print(line)

9158


In [61]:
numCus = SaveCustomerToFile(customer_list)

In [65]:
print(numCus)

8652
